# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_Using the pickup/dropoff lati/longitude to calculate the route distance._

In [4]:
def calculate_distance(from_coord, to_coord):
    """
    Calculate the distance between two coordinates(km)
    return distance in km
    """
    lat1 = from_coord['pickup_latitude'] *pi/180
    lon1 = from_coord['pickup_longitude'] *pi/180
    lat2 = to_coord['dropoff_latitude']*pi/180
    lon2 = to_coord['dropoff_longitude']*pi/180
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = ((dlat/2).map(sin))**2 + (lat1.map(cos)) * (lat2.map(cos)) * ((dlon/2).map(sin))**2
    distance = 6371*2 * (a.map(sqrt)).map(asin)
    
    return round(distance, 2)

In [5]:
def add_distance_column(dataframe):
    """
    Use "pickup_latitude", "pickup_longitude", 
    "dropoff_latitude" and "dropoff_longitude"
    from dataframe to create agrs "from_coord" and "to_coord"
    return distance in km
    """
    from_coord = dataframe[['pickup_latitude', 'pickup_longitude']]
    to_coord = dataframe[['dropoff_latitude', 'dropoff_longitude']]
    dataframe['distance'] = calculate_distance(from_coord, to_coord)

### Processing Taxi Data

_Obtain Yellow Taxi Data from 2009-01 to 2015-06, then clean and combine all dataframes._

In [6]:
"""
Set geopandas coordinates dataframe to match LocationID
"""
import geopandas as gpd
g = gpd.read_file(filename='taxi_zones.zip', engine='fiona')
g = g.to_crs(4326)
g['longitude'] = g.centroid.x
g['latitude'] = g.centroid.y

In [7]:
def find_taxi_csv_urls():
    import re
    y = []
    links = []
    response = requests.get(TAXI_URL)
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")
    yellow = soup.find_all("a", attrs = {"title":"Yellow Taxi Trip Records"})
    y.append([a["href"] for a in yellow])
    
    # use regex pattern to find desired URLs
    pattern = re.compile(".*(2009|201[0-4]|2015-0[1-6]).*")
    for i in y[0]:
        match = re.match(pattern, i)
        if match != None:
            links.append(i)
    return links

In [8]:
def get_and_clean_month_taxi_data(url):
    """
    Obtain data from given months and clean them
    return a dataframe that combines the cleaned data
    """   
    # process those without lat/longitude coordinates
    if 'PULocationID' in df.columns:
        df['pickup_latitude']  = df['PULocationID'].map(g['latitude'])
        df['pickup_longitude'] = df['PULocationID'].map(g['longitude'])
        df['dropoff_latitude'] = df['DOLocationID'].map(g['latitude'])
        df['dropoff_longitude']= df['DOLocationID'].map(g['longitude'])

    # normalizing names
    df = df.rename(columns={'tpep_pickup_datetime':'pickup_datetime', 
                            'Trip_Pickup_DateTime':'pickup_datetime', 
                            'Fare_Amt':'fare_amount',
                            'Start_Lon':'pickup_longitude','Start_Lat':'pickup_latitude', 
                            'End_Lon':'dropoff_longitude', 'End_Lat':'dropoff_latitude',           
                            })
    
    # only keeping columns needed
    df = df[['pickup_datetime','fare_amount',
             'pickup_latitude','pickup_longitude',
             'dropoff_latitude','dropoff_longitude',
            ]]
    # using appropriate column type
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df.iloc[:,1:6] = df.iloc[:,1:6].astype('float32')
        
    # remove invalid ponits
    df = df[df['fare_amount']>0]
        
    # removing trips that start and/or end outside of the BOX
    df = df[df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
        
    df = df[df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
    
    #sampling from data
    df = df.sample(n=round(195454/78))
    
    # return the cleaned month data
    file = url.split('/')[-1]
    print(f'done with {file}')
    return df

In [9]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_parquet = find_taxi_csv_urls()
    for i in all_parquet:
        # maybe: first try to see if you've downloaded this exact file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(i)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data

_Processing the Uber Data, make the type and column names the same as Taxi Data._

In [10]:
def load_and_clean_uber_data(csv_file):
    """
    the same procedure as cleaning taxi data
    we aim to obtain dataframes in the same pattern
    """
    df = pd.read_csv(csv_file)
    # remove unnecessary columns
    df = df[['pickup_datetime','fare_amount',
             'pickup_latitude','pickup_longitude',
             'dropoff_latitude','dropoff_longitude',
            ]]
    # removing trips that start and/or end outside of the BOX
    df = df[df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
        
    df = df[df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
    
    # using appropriate column type
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df.iloc[:,1:6] = df.iloc[:,1:6].astype('float32')
        
    # remove invalid ponits
    df = df[df['fare_amount']>0]
    
    return df

In [11]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplemented()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplemented()

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)